## Usage

Run the `report_historic_ocurrences` to either create or update OcurrenceReports for the end of every day since Aug 8 2020 til today.

In [57]:
import arrow
from datetime import datetime
from typing import List

from apps.patterns.git import repo_at
from apps.patterns.models import OcurrenceReport
from apps.patterns.analyzer import analyze_repo_with_pattern
from apps.utils import to_arrow

def get_end_of_day(arrow_date, days_ago=1, tz="America/Santiago"):
    return arrow_date.shift(days=-days_ago).replace(hour=23, minute=59, second=59, tzinfo=tz)

def report_historic_ocurrences(study_app: str, ignore_paths: List[str] = []):
    now = arrow.now()
    max_datetime = arrow.Arrow(year=2020, month=8, day=15, hour=0, mimute=0, second=0)
    days_left_to_analyze = (now - max_datetime).days
    last_of_days_commits = []
    with repo_at(commit=None, force_delete=False, always_pull=True) as repo:
        current_day = 1
        skip_commits = 0
        max_page = 1000
        commits = repo.iter_commits("devel", max_count=max_page, skip=skip_commits)
        while current_day != days_left_to_analyze:
            critic_moment = get_end_of_day(now, days_ago=current_day)

            last_day_commit_found = False
            while not last_day_commit_found:
                try:
                    commit = next(commits)
                except StopIteration:
                    skip_commits += max_page
                    commits = repo.iter_commits("devel", max_count=max_page, skip=skip_commits)
                    commit = next(commits)

                commit_datetime = to_arrow(commit.committed_datetime)
                if commit_datetime < critic_moment:
                    last_of_days_commits.append(commit)
                    print(f"added new study commit for {commit_datetime}")
                    last_day_commit_found = True

            current_day += 1

    for commit in last_of_days_commits:
        try:
            report = OcurrenceReport.objects.filter(commit=commit.hexsha, app_name=study_app).get()
            report.last_of_day = True
            report.save()
            print(f"Found stored commit {commit.hexsha}")
        except OcurrenceReport.DoesNotExist:
            print(f"Moving repo and analyzing {commit.hexsha}")
            with repo_at(commit=commit.hexsha, force_delete=False, always_pull=True) as repo:
                hits = analyze_repo_with_pattern(pattern=f"apps.{study_app}.models", ignore_paths=ignore_paths)
                payload = dict(
                    app_name=study_app,
                    ocurrences=hits,
                    commit=commit.hexsha,
                    commited_epoch=to_arrow(commit.committed_datetime).timestamp,
                    last_of_day=True
                )
                OcurrenceReport.objects.create(**payload)

                
## Study Catalog
report_historic_ocurrences(study_app="catalog", ignore_paths=["apps/catalog", "apps/promotions", "apps/stores"])

## Study Orders
report_historic_ocurrences(study_app="orders", ignore_paths=["apps/orders"])

added new study commit for 2020-11-08T23:46:09-03:00
added new study commit for 2020-11-06T23:26:47-03:00
added new study commit for 2020-11-06T23:09:40-03:00
added new study commit for 2020-11-05T23:12:32-03:00
added new study commit for 2020-11-04T21:19:03-03:00
added new study commit for 2020-11-03T23:21:24-03:00
added new study commit for 2020-11-02T23:59:00-03:00
added new study commit for 2020-11-01T19:28:52-03:00
added new study commit for 2020-10-31T15:22:36-03:00
added new study commit for 2020-10-30T22:35:07-03:00
added new study commit for 2020-10-29T23:48:54-03:00
added new study commit for 2020-10-28T23:14:06-03:00
added new study commit for 2020-10-27T23:47:02-03:00
added new study commit for 2020-10-26T23:02:43-03:00
added new study commit for 2020-10-25T22:16:15-03:00
added new study commit for 2020-10-23T21:45:55-03:00
added new study commit for 2020-10-23T21:30:21-03:00
added new study commit for 2020-10-22T22:26:40-03:00
added new study commit for 2020-10-21T21:42:52

Found stored commit ed6bdeefb6bd86c1675456da842c1f76af2b323c
Found stored commit 9b5c9ab24e3c5fd5f160571fa2a27590a10383de
Found stored commit c5e69d0068c1c681df50cee6b75d549de28a56a3
Found stored commit 1ccf46ccf598377bc4f1d9752e52b05766963fb2
Found stored commit 3c5637d8218cb4c6936d85d1827dd14ab1b18e58
Found stored commit fb7965d827120293a25e9140d37178621345ec8d
Moving repo and analyzing f072c4896a1d461c928e387998f6eb447fb95057
Found stored commit b395991e3b60ceedcd1245c491c135f87b43ce7a
Found stored commit c1cdba37abfaa7f2ca739f7c543ec965ce7d7086
Found stored commit 0800eea3cb4d884baf2248bf5eb606786ce7b7fd
Found stored commit a315ac00b3687e7e36811170a8bd546821c6f66c
Found stored commit af02b6fa8f3dbe036c476701bf2cbc96e73504b6
Found stored commit 14cc815b0944dc42b5ee7823a2bc060e1ca69fad
Moving repo and analyzing d5149e744c867b7a0dafd65e1b859cce348fb2f7
Found stored commit 357a161747f84b44fc273c6d0d6f833df0d50029
Found stored commit abb79eb34a001e5a735c8ec83295f96aafe2243d
Found stored

Found stored commit bcd9888a6d9c6878e9b1eaac6d3bc0e8ebba8038
Found stored commit ca6dbda8e95985afac81890de6f03f643541e529
Found stored commit cdec7b344a44b0d16e9d0dab204d27850c405801
Found stored commit 6a0a0a6a809cdf969e48584bfccaf850b3a05ecb
Found stored commit cfe593543f84e99df977abc434474687927bfdd2
Found stored commit 92e387b5fb21f82d7aedf999ccc8a2dba4e9b271
Moving repo and analyzing 23747cb61df8d828fcf403310a3e28806c78ef5c
Found stored commit 5fccb8c3961dfb9ce6ffa040ce3768175960874b
Found stored commit 4861b26aa2e142f00670e1225af9f9c19c3f379a
Found stored commit 89926ef50f5e46780d000005a615f4ae45dd32f1
Found stored commit 6d78fb3f593a59341828deb3165c9f1fa4a96e65
Found stored commit 1fc51400777cda7608b1dfca8ad0d2bf20ab1aa2
Found stored commit 15e0407494f072cad2a8176b22e64a5acd66e0d8
Found stored commit 09f0acd230877749e17ab629809ff2348f4c75cf
Found stored commit 032655c5e8b7a4756e6a75db615d19ef1c48f9a2
Moving repo and analyzing 9b96c6b683f1b0c9a9e72c1b1f16164f2258ad5b
Found stored

In [63]:
import arrow
all_reports = OcurrenceReport.objects.filter(last_of_day=True, app_name="orders").order_by('commited_epoch').values_list('_total', 'commited_epoch')
for tot, epoch in all_reports:
    print(arrow.get(epoch).format('YYYY-MM-DD'))
for tot, epoch in all_reports:
    print(tot)

2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-31
2020-09-01
2020-09-02
2020-09-03
2020-09-04
2020-09-05
2020-09-07
2020-09-08
2020-09-09
2020-09-10
2020-09-11
2020-09-12
2020-09-13
2020-09-15
2020-09-16
2020-09-17
2020-09-18
2020-09-20
2020-09-21
2020-09-22
2020-09-23
2020-09-24
2020-09-25
2020-09-26
2020-09-27
2020-09-29
2020-09-30
2020-10-01
2020-10-02
2020-10-03
2020-10-04
2020-10-06
2020-10-07
2020-10-08
2020-10-09
2020-10-10
2020-10-11
2020-10-13
2020-10-14
2020-10-15
2020-10-16
2020-10-17
2020-10-18
2020-10-19
2020-10-20
2020-10-21
2020-10-22
2020-10-23
2020-10-24
2020-10-25
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31
2020-11-01
2020-11-03
2020-11-04
2020-11-05
2020-11-06
2020-11-07
2020-11-09
590
592
592
591
595
597
597
592
600
599
600
599
602
602
602
603
603
606
608
607
607
607
607
611
612
612
611
612
611
611
611
611
613
619
623
625
630
630
625
631
632


In [56]:
from apps.patterns.models import OcurrenceReport
rep = OcurrenceReport.objects.filter(last_of_day=True, app_name="catalog").order_by("-commited_epoch").values_list('_total', 'commited_epoch').first()
#rep.total

print(rep)
print(arrow.get(rep[1]).format('YYYY-MM-DD'))
print(rep[0])

(292, 1604269732)
2020-11-01
292


# Clean ocurrences

Sometimes more than one OcurrenceReport with last_of_day=True is created for the same day. We should only keep the "latest" one

In [60]:
from apps.patterns.models import OcurrenceReport


reports = OcurrenceReport.objects.filter(last_of_day=True, app_name="orders")
acc = {}

for report in reports:
    report_date = arrow.get(report.commited_epoch).date()
    if report_date in acc:
        element = acc[report_date]
        element.append((report.id, report.commited_epoch))
        acc[report_date] = element
    else:
        acc[report_date] = [(report.id, report.commited_epoch)]

ids_to_delete = []
for date, metadata in acc.items():
    if len(metadata) > 1:
        ordered_metadata = sorted(metadata, key=lambda x: x[1], reverse=True)
        all_but_first = ordered_metadata[1:]
        ids_to_delete.extend(list(map(lambda x: x[0], all_but_first)))

print(ids_to_delete)
OcurrenceReport.objects.filter(id__in=ids_to_delete).delete()

[1043, 1044, 1045, 1047, 1048, 1049, 1046, 1041, 877, 1040, 1035]


(11, {'patterns.OcurrenceReport': 11})